# Offline Engine API

SGLang provides a direct inference engine without the need for an HTTP server, especially for use cases where additional HTTP server adds unnecessary complexity or overhead. Here are two general use cases:

- Offline Batch Inference
- Custom Server on Top of the Engine

This document focuses on the offline batch inference, demonstrating four different inference modes:

- Non-streaming synchronous generation
- Streaming synchronous generation
- Non-streaming asynchronous generation
- Streaming asynchronous generation

Additionally, you can easily build a custom server on top of the SGLang offline engine. A detailed example working in a python script can be found in [custom_server](https://github.com/sgl-project/sglang/blob/main/examples/runtime/engine/custom_server.py).

## Offline Batch Inference

SGLang offline engine supports batch inference with efficient scheduling.

In [1]:
# launch the offline engine

import sglang as sgl
import asyncio

llm = sgl.Engine(model_path="meta-llama/Meta-Llama-3.1-8B-Instruct")

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]


Loading safetensors checkpoint shards:  25% Completed | 1/4 [00:00<00:00,  4.84it/s]


Loading safetensors checkpoint shards:  50% Completed | 2/4 [00:01<00:01,  1.77it/s]


Loading safetensors checkpoint shards:  75% Completed | 3/4 [00:01<00:00,  1.34it/s]


Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:02<00:00,  1.20it/s]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:02<00:00,  1.36it/s]



### Non-streaming Synchronous Generation

In [2]:
prompts = [
    "Hello, my name is",
    "The president of the United States is",
    "The capital of France is",
    "The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

outputs = llm.generate(prompts, sampling_params)
for prompt, output in zip(prompts, outputs):
    print("===============================")
    print(f"Prompt: {prompt}\nGenerated text: {output['text']}")

Prompt: Hello, my name is
Generated text:  Marie and I’m a freelance writer based in the beautiful countryside of rural France. I specialise in writing travel articles, blog posts, and copy for the travel and tourism industry. I’m also passionate about creating engaging and informative content for websites, social media, and publications that inspire people to explore new destinations and experiences.

With over 10 years of experience in writing and travel, I have developed a unique voice and style that brings destinations to life. My writing is descriptive, engaging, and informative, making it perfect for travel enthusiasts, tour operators, and travel companies looking to showcase their products and services.

My areas of expertise include:

*
Prompt: The president of the United States is
Generated text:  the head of state and head of government of the United States. The president leads the executive branch of the federal government and is the commander-in-chief of the United States A

### Streaming Synchronous Generation

In [3]:
prompts = [
    "Hello, my name is",
    "The capital of France is",
    "The future of AI is",
]
sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing synchronous streaming generation ===")

for prompt in prompts:
    print(f"\nPrompt: {prompt}")
    print("Generated text: ", end="", flush=True)

    for chunk in llm.generate(prompt, sampling_params, stream=True):
        print(chunk["text"], end="", flush=True)
    print()


=== Testing synchronous streaming generation ===

Prompt: Hello, my name is
Generated text: 

 Maria

 and

 I

'm

...

 tasks

 and

 enabling

 us

 to

### Non-streaming Asynchronous Generation

In [4]:
prompts = [
    "Hello, my name is",
    "The capital of France is",
    "The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing asynchronous batch generation ===")


async def main():
    outputs = await llm.async_generate(prompts, sampling_params)

    for prompt, output in zip(prompts, outputs):
        print(f"\nPrompt: {prompt}")
        print(f"Generated text: {output['text']}")


asyncio.run(main())


=== Testing asynchronous batch generation ===



Prompt: Hello, my name is
Generated text:  Daniel. I am a mechanical engineer. My main area of expertise is vibration analysis and structural dynamics. I have worked in the aerospace industry for many years and have written several papers on the subject of vibration testing and analysis. I have also taught classes on vibration analysis and structural dynamics at a university. I am interested in speaking with someone about a potential collaboration on a project involving vibration analysis of a complex structure. Can you tell me a little bit about your background and how you think you might be able to contribute to this project? Thank you.
Daniel,
It sounds like you have a strong background in vibration analysis and structural dynamics, particularly in the

Prompt: The capital of France is
Generated text:  Paris, a city of romance, fashion and fine cuisine. Paris is one of the world's top tourist destinations, attracting over 23 million visitors each year. It is known as the City of Li

### Streaming Asynchronous Generation

In [5]:
prompts = [
    "Hello, my name is",
    "The capital of France is",
    "The future of AI is",
]
sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing asynchronous streaming generation ===")


async def main():
    for prompt in prompts:
        print(f"\nPrompt: {prompt}")
        print("Generated text: ", end="", flush=True)

        generator = await llm.async_generate(prompt, sampling_params, stream=True)
        async for chunk in generator:
            print(chunk["text"], end="", flush=True)
        print()


asyncio.run(main())


=== Testing asynchronous streaming generation ===

Prompt: Hello, my name is
Generated text: 

 Rachel

.

 I

 am

...

 and

 user

-friendly

,

 making

In [6]:
llm.shutdown()